### Part 1 : Introduction and Data Sections
##### I. Introduction

##### Problem background:
Pizza is a food that is easy, you will just pick up your phone, and call the nearest pizza place. However, due to the recent pandemic some pizza places were closed down. especially in new york where majority of the US cases is located. I want to determine which pizza places deliver food during these pandemic. 

With the facilitating limitations, it is critical to recognize how Pizza fans can securely stuff their countenances. Ideally is discover a Pizza eatery that does food delivery. This is get the most sweltering pizza, while keeping up detachment at home. 



                          
##### Data Description
For this investigation I am posting down the sources and sorts of information I would utilize CDC COVID information postponed neighborhood insightful. NYC civil information in geoJson group enumerating the area geo information. Foursquare information for Pizza eatery areas. I would do a minor cleanup since the information is from various sources. Key here would be the Modified CTA is an adjusted ZIP code information.
https://github.com/nychealth/coronavirus-information

